## Perovskite Workflow 1 Client setup

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib

In [2]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

In [7]:
# Get the Haverford college lab
haverford_lab_response = client.get(endpoint='actor', data={'description': 'Haverford College', 'person__isnull': True, 'systemtool__isnull': True})[0]

{'description': 'Haverford College', 'person__isnull': True, 'systemtool__isnull': True}
http://localhost:8000/api/actor/?description=Haverford+College&person__isnull=True&systemtool__isnull=True
GET: OK. Found 1 results


## Creating a template
First, we specify a name and create and entry for a template

In [8]:
template_response = client.get_or_create(endpoint='experiment-template', data={'description': 'Workflow 1', 'ref_id': 'workflow_1', 'lab': haverford_lab_response['url']})[0]

{'description': 'Workflow 1', 'ref_id': 'workflow_1', 'lab': '635bc678-5f2a-4220-b21e-505d86154283'}
http://localhost:8000/api/experiment-template/?description=Workflow+1&ref_id=workflow_1&lab=635bc678-5f2a-4220-b21e-505d86154283
GET: OK. Found 1 results


Then we specify templates for reagents that will be used in the experiment. A reagent template requires a name and type(s) of chemicals associated with that reagent.
For the Workflow 1 template, and example of a reagent name is "Reagent 1 - Solvent" and it's chemical type is "solvent". Note that reagent name can be any arbitrary string but the chemical type must exist in the escalate system prior to specifying the reagent template.

### Specifying Reagent 1

In [3]:
# Create the reagent template. 
reagent_template_data = {"description": "Reagent 1 - Solvent"}
reagent_1_response = client.get_or_create(endpoint='reagent-template', data=reagent_template_data)[0]

# Types of chemicals in reagent. 
chemical_type_data = {"description": "solvent"}
chemical_type_response = client.get_or_create(endpoint='material-type', data=chemical_type_data)[0]

# Associate the chemical type with the reagent. A reagent can have multiple chemicals with different types. In this case there is only 1 solvent
reagent_material_data = {"description": "Reagent 1 - Solvent: solvent", "reagent_template": reagent_1_response['url'], "material_type": chemical_type_response['url']}
reagent_material_1_response = client.get_or_create(endpoint='reagent-material-template', data=reagent_material_data)

{'description': 'Reagent 1 - Solvent'}
http://localhost:8000/api/reagent-template/?description=Reagent+1+-+Solvent
GET: OK. Found 1 results
{'description': 'solvent'}
http://localhost:8000/api/material-type/?description=solvent
GET: OK. Found 1 results
{'description': 'Reagent 1 - Solvent: solvent', 'reagent_template': '6a06ebf6-b72a-45c3-9e50-d4fafe6d2577', 'material_type': '72170fe7-c7bf-4dd8-a90c-d0db9404e368'}
http://localhost:8000/api/reagent-material-template/?description=Reagent+1+-+Solvent%3A+solvent&reagent_template=6a06ebf6-b72a-45c3-9e50-d4fafe6d2577&material_type=72170fe7-c7bf-4dd8-a90c-d0db9404e368
GET: OK. Found 1 results


## Specifying multiple reagents
Usually an experiment has multiple reagents, Escalate client provides a convenience function to add multiple reagent templates 

In [3]:
# Reagent templates can be defined as a dictionary of reagent template name as keys and a list of chemical types as value
# If the chemical type does not exist, it is automatically created 
reagent_templates = {
            "Reagent 1 - Solvent": ["solvent"],
            "Reagent 2 - Stock A": ["organic", "solvent"],
            "Reagent 3 - Stock B": ["inorganic", "organic", "solvent"],
            "Reagent 7 - Acid": ["acid"],
        }

reagent_template_responses = client.create_reagent_templates(data=reagent_templates)

{'description': 'Reagent 1 - Solvent'}
http://localhost:8000/api/reagent-template/?description=Reagent+1+-+Solvent
GET: OK. Found 1 results
{'description': 'solvent'}
http://localhost:8000/api/material-type/?description=solvent
GET: OK. Found 1 results
{'description': 'Reagent 1 - Solvent: solvent', 'reagent_template': '6a06ebf6-b72a-45c3-9e50-d4fafe6d2577', 'material_type': '72170fe7-c7bf-4dd8-a90c-d0db9404e368'}
http://localhost:8000/api/reagent-material-template/?description=Reagent+1+-+Solvent%3A+solvent&reagent_template=6a06ebf6-b72a-45c3-9e50-d4fafe6d2577&material_type=72170fe7-c7bf-4dd8-a90c-d0db9404e368
GET: OK. Found 1 results
{'description': 'Reagent 2 - Stock A'}
http://localhost:8000/api/reagent-template/?description=Reagent+2+-+Stock+A
GET: OK. Found 1 results
{'description': 'organic'}
http://localhost:8000/api/material-type/?description=organic
GET: OK. Found 1 results
{'description': 'Reagent 2 - Stock A: organic', 'reagent_template': '263ec3d8-30a2-4a4b-8623-e282899f6b

## Default values
We can associate certain templates with default values. These values are used to automatically pre-populate values, their types and measurement units.
For example, here we create a "Zero ml" default value that is used to pre-populates liquid chemical reagent volumes with the value "0 milliliters"

In [4]:
volume_value = {"value": 0, "unit": "ml", "type": "num"}
zero_ml_data = {"description": "Zero ml", "nominal_value": volume_value, "actual_value": volume_value,}
zero_ml_response = client.get_or_create(endpoint='default-values', data=zero_ml_data)[0]

{'description': 'Zero ml'}
http://localhost:8000/api/default-values/?description=Zero+ml
GET: OK. Found 12 results


## Properties of Reagents and Reagent Materials
One can associate properties and values to reagents as well as materials within a reagent. For example, we can capture the total volume and dead volume for 'Reagent 1 - Solvent' and separately capture values such as concentration and amount for each chemical component in the reagent. In the case of 'Reagent 1 - Solvent' we only have 1 component of type solvent

In [5]:
# Defining Reagent Template properties of total reagent volume and dead volume and associate a default volume
total_volume_prop_template_data  = {
                "description": "total volume",
                "property_def_class": "extrinsic",
                "short_description": "volume",
                "default_value": zero_ml_response['url'],
            }
total_volume_prop_template = client.get_or_create(endpoint='property-template', data=total_volume_prop_template_data)[0]

dead_volume_prop_template_data = {
                "description": "dead volume",
                "property_def_class": "extrinsic",
                "short_description": "dead volume",
                "default_value": zero_ml_response['url'],
            }
dead_volume_prop_template = client.get_or_create(endpoint='property-template', data=dead_volume_prop_template_data)[0]

# Assigning total volume and dead volume properties to "Reagent 1 - Solvent"
data = {'properties':[total_volume_prop_template['url'], dead_volume_prop_template['url']]}
client.patch(endpoint='reagent-template', resource_id=reagent_template_responses['Reagent 1 - Solvent']['uuid'] ,data=data)

{'description': 'total volume', 'property_def_class': 'extrinsic', 'short_description': 'volume', 'default_value': '2318afaa-62d3-4d1c-965f-be918c10cf05'}
http://localhost:8000/api/property-template/?description=total+volume&property_def_class=extrinsic&short_description=volume&default_value=2318afaa-62d3-4d1c-965f-be918c10cf05
GET: OK. Found 1 results
{'description': 'dead volume', 'property_def_class': 'extrinsic', 'short_description': 'dead volume', 'default_value': '2318afaa-62d3-4d1c-965f-be918c10cf05'}
http://localhost:8000/api/property-template/?description=dead+volume&property_def_class=extrinsic&short_description=dead+volume&default_value=2318afaa-62d3-4d1c-965f-be918c10cf05
GET: OK. Found 1 results
Status 200: OK


{'url': 'http://localhost:8000/api/reagent-template/6a06ebf6-b72a-45c3-9e50-d4fafe6d2577/',
 'uuid': '6a06ebf6-b72a-45c3-9e50-d4fafe6d2577',
 'edocs': [],
 'tags': [],
 'notes': [],
 'add_date': '2022-02-17T18:00:10.221478',
 'mod_date': '2022-02-23T03:00:48.056753',
 'description': 'Reagent 1 - Solvent',
 'internal_slug': 'reagent-1-solvent',
 'status': None,
 'properties': ['http://localhost:8000/api/property-template/6f5dae35-4b20-4eb0-aa00-352a4eae132c/',
  'http://localhost:8000/api/property-template/d0dd104e-a95f-453a-bf74-8a38b1702762/']}

In [14]:
# Get chemical
reagent_1_solvent = client.get(endpoint='reagent-template', 
                               resource_id=reagent_template_responses['Reagent 1 - Solvent']['uuid'], 
                               related_endpoint='reagent-material-template')[0]
# Default values
zero_conc_value = {"value": 0, "unit": "M", "type": "num"}
zero_conc_data = {"description": "Zero M", "nominal_value": zero_conc_value, "actual_value": zero_conc_value,}
zero_conc_response = client.get_or_create(endpoint='default-values', data=zero_conc_data)[0]

zero_g_value = {"value": 0, "unit": "g", "type": "num"}
zero_g_data = {"description": "Zero g", "nominal_value": zero_g_value, "actual_value": zero_g_value,}
zero_g_response = client.get_or_create(endpoint='default-values', data=zero_g_data)[0]

# Add concentration
data = {"description": "concentration",
        "reagent_material_template": reagent_1_solvent['url'], # Associating it with solvent material
        "default_value": zero_conc_response['url']}

client.get_or_create('reagent-material-value-template', data=data)

# Add amounts
data = {"description": "amount",
        "reagent_material_template": reagent_1_solvent['url'], # Associating it with solvent material
        "default_value": zero_g_response['url']}

client.get_or_create('reagent-material-value-template', data=data)

{}
http://localhost:8000/api/reagent-template/6a06ebf6-b72a-45c3-9e50-d4fafe6d2577/reagent-material-template/
GET: OK. Found 1 results
{'description': 'Zero M'}
http://localhost:8000/api/default-values/?description=Zero+M
GET: OK. Found 1 results
{'description': 'Zero g'}
http://localhost:8000/api/default-values/?description=Zero+g
GET: OK. Found 1 results
{'description': 'concentration', 'reagent_material_template': '25987ad6-3728-4691-a5bc-77fc051af132', 'default_value': '564f3a82-6395-4516-a6ac-8bf9d82898ea'}
http://localhost:8000/api/reagent-material-value-template/?description=concentration&reagent_material_template=25987ad6-3728-4691-a5bc-77fc051af132&default_value=564f3a82-6395-4516-a6ac-8bf9d82898ea
GET: OK. Found 1 results
{'description': 'amount', 'reagent_material_template': '25987ad6-3728-4691-a5bc-77fc051af132', 'default_value': 'b8fcdb40-9879-40f6-a71d-10eda9a84fe6'}
http://localhost:8000/api/reagent-material-value-template/?description=amount&reagent_material_template=25

[{'url': 'http://localhost:8000/api/reagent-material-value-template/64fcd2c3-2089-4454-b96a-64447b61dae6/',
  'uuid': '64fcd2c3-2089-4454-b96a-64447b61dae6',
  'edocs': [],
  'tags': [],
  'notes': [],
  'add_date': '2022-02-17T18:00:10.243081',
  'mod_date': '2022-02-17T18:00:10.243091',
  'description': 'amount',
  'status': None,
  'reagent_material_template': 'http://localhost:8000/api/reagent-material-template/25987ad6-3728-4691-a5bc-77fc051af132/',
  'default_value': 'http://localhost:8000/api/default-values/b8fcdb40-9879-40f6-a71d-10eda9a84fe6/'}]

In [15]:
# Convenience function to add all reagent-material-value-templates here

## Outcome Template
Outcome template of an experiment requires a description, an experiment template and an optional default value, in this experiment we collect crystal scores represented by integers

In [9]:
# Create outcome template
data = {'description': 'Crystal score', 'experiment': template_response['url']}
outcome_response = client.get_or_create('outcome-template', data=data)

{'description': 'Crystal score', 'experiment': '489c1000-b4ae-499c-bc56-82906924e1df'}
http://localhost:8000/api/outcome-template/?description=Crystal+score&experiment=489c1000-b4ae-499c-bc56-82906924e1df
GET: OK. Found 1 results


## Action Sequences
A series of actions strung together in order constitutes an action sequence. Action sequences once created, can be re-used in other experiment templates
In this template, we create action sequences that contain only one action

In [22]:
# List of action sequence names
action_sequence_names = ["Preheat Temperature (C)", "Mixing time1 (s)",
            "Mixing time2 (s)", "Temperature (C)", "Stir Rate (rpm)",
            "Reaction time (s)", "Dispense Reagent 1 - Solvent", "Dispense Reagent 2 - Stock A"
            "Dispense Reagent 3 - Stock B", "Dispense Reagent 7 - Acid Volume 1",
            "Dispense Reagent 7 - Acid Volume 2"]

# Creating action sequences and associating them with the experiment template
asn_responses = {}
for sequence_number, asn in enumerate(action_sequence_names):
    data = {'description': asn}
    # Create action sequence
    asn_responses[asn] = client.get_or_create(endpoint='action-sequence', data=data)[0]
    # Associate with experiment template
    data = {'experiment_action_sequence_seq': sequence_number,
            'experiment_template': template_response['url'],
            'action_sequence': asn_responses[asn]['url']}
    client.get_or_create(endpoint='experiment-action-sequence', data=data)


{'description': 'Preheat Temperature (C)'}
http://localhost:8000/api/action-sequence/?description=Preheat+Temperature+%28C%29
GET: OK. Found 1 results
{'experiment_template': '489c1000-b4ae-499c-bc56-82906924e1df', 'action_sequence': 'c7accacf-ca99-438f-af03-12319e04ad13'}
http://localhost:8000/api/experiment-action-sequence/?experiment_template=489c1000-b4ae-499c-bc56-82906924e1df&action_sequence=c7accacf-ca99-438f-af03-12319e04ad13
GET: OK. Found 2 results
{'description': 'Mixing time1 (s)'}
http://localhost:8000/api/action-sequence/?description=Mixing+time1+%28s%29
GET: OK. Found 1 results
{'experiment_action_sequence_seq': 1, 'experiment_template': '489c1000-b4ae-499c-bc56-82906924e1df', 'action_sequence': '7da78111-7099-4cd1-9217-b3cc4fa947ec'}
http://localhost:8000/api/experiment-action-sequence/?experiment_action_sequence_seq=1&experiment_template=489c1000-b4ae-499c-bc56-82906924e1df&action_sequence=7da78111-7099-4cd1-9217-b3cc4fa947ec
GET: OK. Found 1 results
{'description': 'M

## Creating actions
Pre-requisites to creating actions are: Action Definition, Action Parameter Definition, Source material and Destination material

### Action definition
Action definitions are templates for actions that can be re-used in other experiment templates. For this template we require 3 action definitions: `dispense`, `bring_to_temperature` and `stir`

In [4]:
# Create a dispense action definition
data = {'description': 'dispense'}
dispense_action_def_response = client.get_or_create(endpoint='action-def', data=data)[0]

# This action definition stores the volume to be dispensed defined as a parameter definition
data = {'description': 'volume', 'default_value': {"type": "num", "unit": "mL", "value": 0.0}}
volume_parameter_def_response = client.get_or_create(endpoint='parameter-def', data=data)[0]

action_parameter_def = {
            "dispense": [("volume",{"type": "num", "unit": "mL", "value": 0.0})],
            "bring_to_temperature": [("temperature", {"type": "num", "unit": "degC", "value": 0.0})],
            "stir": [("duration", {"type": "num", "unit": "seconds", "value": 0.0}),  ("speed", {"type": "num", "unit": "rpm", "value": 0.0})],
        }
# Convenience function to add action defs and their parameter definitions
action_defs, parameter_defs = client.create_action_parameters(data=action_parameter_def)


{'description': 'dispense'}
http://localhost:8000/api/action-def/?description=dispense
GET: OK. Found 1 results
{'description': 'volume'}
http://localhost:8000/api/parameter-def/?description=volume
GET: OK. Found 2 results
{'description': 'dispense'}
http://localhost:8000/api/action-def/?description=dispense
GET: OK. Found 1 results
[('volume', {'type': 'num', 'unit': 'mL', 'value': 0.0})]
{'description': 'volume'}
http://localhost:8000/api/parameter-def/?description=volume
GET: OK. Found 2 results
{'description': 'bring_to_temperature'}
http://localhost:8000/api/action-def/?description=bring_to_temperature
GET: OK. Found 1 results
[('temperature', {'type': 'num', 'unit': 'degC', 'value': 0.0})]
{'description': 'temperature'}
http://localhost:8000/api/parameter-def/?description=temperature
GET: OK. Found 2 results
{'description': 'stir'}
http://localhost:8000/api/action-def/?description=stir
GET: OK. Found 1 results
[('duration', {'type': 'num', 'unit': 'seconds', 'value': 0.0}), ('spe

## Actions
Next we add specific actions to action sequences

In [ ]:
# Preheat plate
action